In [1]:
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [2]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from datetime import datetime
import keras
import glob
import random
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import Conv1D, MaxPooling1D

def load_data():
    filename_queue=glob.glob('./rawdata/data[1-4].csv')
    
    dataX = []
    dataY = []
    trainX=[]
    testX=[]
    trainY=[]
    testY=[]
    label_count=np.zeros(16)

    for filename in filename_queue:
        file=open(filename, newline='')
        reader=csv.reader(file)
        header=next(reader)
        read = list(reader)
        random.shuffle(read)

        for row in read:
            row[4:]=[float(i) for i in row[4:]]
            date=datetime.strptime(row[0],'%Y/%m/%d')
            label=int(row[3])
            label_count[label]+=1

            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([X_acc[0:150],Y_acc[0:150],Z_acc[0:150], X_gyro[0:150], Y_gyro[0:150], Z_gyro[0:150]])
                window = np.transpose(window)

                if (label_count[label]%5!=1):
                    trainX.append(window)
                    trainY.append(label)
                else:
                    testX.append(window)
                    testY.append(label)
    
    X_train = np.stack(trainX, axis = 0)
    y_train = np.stack(trainY, axis = 0)
    X_test = np.stack(testX, axis = 0)
    y_test = np.stack(testY, axis = 0)
    X_train.reshape(-1,900)
    #one-hot encode
    #y_train = to_categorical(y_train)
    #y_test = to_categorical(y_test)
#     print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    return X_train, y_train, X_test, y_test

(X_train, y_train, X_test, y_test)=load_data()
print(y_test)


Using TensorFlow backend.


[ 9  9  9  9 12 12 12 12 10 10 10 10  9  9  9  9 10 10 10 10 12 12 12 12
  9  9  9  9  6  6  6  6 10 10 10 10 10 10 10 10  0  0  0  0  7  7  7  7
  1  1  1  1  3  3  3  3 13 13 13 13  2  2  2  2 12 12 12 12  5  5  5  5
 14 14 14 14  0  0  0  0  1  1  1  1  6  6  6  6  8  8  8  8 12 12 12 12
  9  9  9  9  7  7  7  7  6  6  6  6  5  5  5  5  0  0  0  0 12 12 12 12
  1  1  1  1  6  6  6  6 13 13 13 13 10 10 10 10  0  0  0  0  6  6  6  6
 12 12 12 12  6  6  6  6  5  5  5  5  8  8  8  8 12 12 12 12 14 14 14 14
  4  4  4  4  2  2  2  2  3  3  3  3  0  0  0  0 12 12 12 12  6  6  6  6
  5  5  5  5  1  1  1  1  7  7  7  7 13 13 13 13 12 12 12 12 12 12 12 12
  0  0  0  0  8  8  8  8  6  6  6  6 12 12 12 12  5  5  5  5  3  3  3  3
 12 12 12 12  5  5  5  5  1  1  1  1  8  8  8  8  6  6  6  6 12 12 12 12]


In [3]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
]

# Output classes to learn how to classify
LABELS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
    
]

In [4]:
print(len(X_train[0][0]))

6


In [5]:
training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 150 timesteps per series
n_input = len(X_train[0][0])  # 6 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 16 # Total classes 


# Training

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 100  # Loop 300 times on the dataset
batch_size = 150
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(264, 150, 6) (264,) -0.9872823265909091 37.91252399917788
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [6]:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input])
    # new shape: (n_steps*batch_size, n_input)

    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data.

    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index]

    return batch_s


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes
    # e.g.:
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [7]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs,
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)

    # Evaluate network only at some steps for faster training:
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):

        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))

        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy],
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

Training iter #150:   Batch Loss = 4.697142, Accuracy = 0.0
PERFORMANCE ON TEST SET: Batch Loss = 4.876418590545654, Accuracy = 0.022727273404598236
Training iter #30000:   Batch Loss = 0.964210, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 3.117014169692993, Accuracy = 0.560606062412262
Training iter #60000:   Batch Loss = 0.744918, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 3.2538959980010986, Accuracy = 0.5454545617103577
Training iter #90000:   Batch Loss = 0.655190, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 3.1202664375305176, Accuracy = 0.5416666865348816
Optimization Finished!
FINAL RESULT: Batch Loss = 3.115935802459717, Accuracy = 0.5378788113594055
